In [1]:
import sys
!{sys.executable} -m pip install kagglehub

In [28]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("basilb2s/language-detection")

print("Path to dataset files:", path)

Path to dataset files: /Users/dhyanyajnik/.cache/kagglehub/datasets/basilb2s/language-detection/versions/1


In [29]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

import warnings
warnings.simplefilter('ignore')

In [30]:
df = pd.read_csv(path+'/Language Detection.csv')

In [31]:
df.head()

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English


In [32]:
X = df['Text']
y = df['Language']

In [33]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [34]:
le.classes_

array(['Arabic', 'Danish', 'Dutch', 'English', 'French', 'German',
       'Greek', 'Hindi', 'Italian', 'Kannada', 'Malayalam', 'Portugeese',
       'Russian', 'Spanish', 'Sweedish', 'Tamil', 'Turkish'], dtype=object)

In [35]:
data_list = []

for text in X:
    text = re.sub(r'[!@#$(),\n"%^*?\:;~`0-9]', ' ', text)
    text = re.sub(r'[[]]', ' ', text)
    text = text.lower()
    data_list.append(text)

In [36]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit(X_train)

X_train = cv.transform(X_train).toarray()
X_test = cv.transform(X_test).toarray()

In [12]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB()

In [13]:
y_pred = mnb.predict(X_test)

In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

ac = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

In [15]:
print("Accuracy = ", ac)

Accuracy =  0.9763056092843327


In [37]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([('vectorizer', cv), ('multinomialNB', mnb)])
pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('multinomialNB', MultinomialNB())])

In [38]:
y_pred2 = pipe.predict(X_test)
ac2 = accuracy_score(y_test, y_pred2)
print("Accuracy = ", ac2)

Accuracy =  0.9787234042553191


In [39]:
with open('trained_pipeline-0.1.0.pkl', 'wb') as f:
    pickle.dump(pipe, f)

In [ ]:
# !zip -r ./trained_pipeline-0.1.0.pkl.zip ./trained_pipeline-0.1.0.pkl
# if as a folder instead of a file

  adding: trained_pipeline-0.1.0.pkl (deflated 95%)


In [ ]:
text = 'Hello how are you'
y = pipe.predict([text])
le.classes_[y[0]], y

('English', array([3]))

In [ ]:
import spacy

# Load spaCy's small English model
nlp = spacy.load("en_core_web_sm")

# Sample text
text = "The quick brown fox jumps over the lazy dog."

# Process the text
doc = nlp(text)

#Print POS tags for each token
print(f"{'Token':<15} {'POS':<10} {'Tag':<10} {'Explanation'}")
print("-" * 50)
for token in doc:
    print(f"{token.text:<15} {token.pos_:<10} {token.tag_:<10} {spacy.explain(token.tag_)}")


The quick brown fox jumps over the lazy dog.


In [25]:
import spacy

# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

# Sample text
text = "The quick brown fox jumps over the lazy dog."

# Process the text
doc = nlp(text)

# Create dictionary: word -> POS
pos_dict = {token.text: token.pos_ for token in doc if token.is_alpha}

# Print the dictionary
print(pos_dict)


{'The': 'DET', 'quick': 'ADJ', 'brown': 'ADJ', 'fox': 'NOUN', 'jumps': 'VERB', 'over': 'ADP', 'the': 'DET', 'lazy': 'ADJ', 'dog': 'NOUN'}


In [27]:
token.text

'.'